In [13]:
import os
import numpy as np 
import shutil

In [14]:
#載入資料集
dir_data = 'C:/Users/luluhsiao/Documents/GitHub/2nd-ML100Days/ml-marathon-final/data/kaggle_dogcat/train/'
train_dat = os.listdir(dir_data)
train_cat = filter(lambda x:x[:3] == 'cat', train_dat)
train_dog = filter(lambda x:x[:3] == 'dog', train_dat)

PermissionError: [WinError 5] 存取被拒。: 'C:/Users/luluhsiao/Documents/GitHub/2nd-ML100Days/ml-marathon-final/data/kaggle_dogcat/train/'

In [ ]:
train_dat

In [12]:
rmrf_mkdir(dir_data)

PermissionError: [WinError 5] 存取被拒。: 'C:/Users/luluhsiao/Documents/GitHub/2nd-ML100Days/ml-marathon-final/data/kaggle_dogcat/train/'

In [10]:
train_dog

In [30]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from keras import callbacks
from keras.models import Sequential, model_from_yaml, load_model
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.utils import np_utils, plot_model
from sklearn.model_selection import train_test_split
from keras.applications.resnet50 import preprocess_input, decode_predictions

In [31]:
def load_data():
    path = './data/kaggle_dogcat/train/'
    files = os.listdir(path)
    images = []
    labels = []
    for f in files:
        img_path = path + f
        img = image.load_img(img_path, target_size=image_size)
        img_array = image.img_to_array(img)
        images.append(img_array)

        if 'cat' in f:
            labels.append(0)
        else:
            labels.append(1)

    data = np.array(images)
    labels = np.array(labels)

    labels = np_utils.to_categorical(labels, 2)
    return data, labels

In [33]:
# 影像大小
image_size = (224, 224)
images, lables = load_data()
images /= 255
x_train, x_test, y_train, y_test = train_test_split(images, lables, test_size=0.2)

PermissionError: [Errno 13] Permission denied: './data/kaggle_dogcat/train/cats'

In [32]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(5, 5), input_shape=(img_h, img_h, 3), activation='relu', padding='same'))
model.add(MaxPool2D())
model.add(Dropout(0.3))

model.add(Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPool2D())
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPool2D())
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPool2D())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

model.summary() 

NameError: name 'img_h' is not defined

In [ ]:
sgd = Adam(lr=0.0003)
model.compile(loss='binary_crossentropy',optimizer=sgd, metrics=['accuracy'])

In [ ]:
scroe, accuracy = model.evaluate(x_test, y_test, batch_size=nbatch_size)
print('scroe:', scroe, 'accuracy:', accuracy)

In [ ]:
yaml_string = model.to_yaml()
with open('./models/cat_dog.yaml', 'w') as outfile:
    outfile.write(yaml_string)
model.save_weights('./models/cat_dog.h5')

In [ ]:
def pred_data():

    with open('./models/cat_dog.yaml') as yamlfile:
        loaded_model_yaml = yamlfile.read()
    model = model_from_yaml(loaded_model_yaml)
    model.load_weights('./models/cat_dog.h5')

    sgd = Adam(lr=0.0003)
    model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])

    images = []
    path='./data/test/'
    for f in os.listdir(path):
        img = image.load_img(path + f, target_size=image_size)
        img_array = image.img_to_array(img)

        x = np.expand_dims(img_array, axis=0)
        x = preprocess_input(x)
        result = model.predict_classes(x,verbose=0)

        print(f,result[0])